In [1]:
from popframe.method.popuation_frame import PopFrame
from popframe.models.region import Region
region_model = Region.from_pickle('/Users/mvin/Code/PopFrame/examples/data/data_models/Ленинградская область.pickle')

In [2]:
visualizer = PopFrame(region=region_model)
output = visualizer.build_circle_frame(output_type='html')  # or 'gdf'
output

'HTML map saved as final_circle.html'

In [3]:
from popframe.method.popuation_frame import PopFrame
visualizer = PopFrame(region=region_model)
output = visualizer.build_circle_frame(output_type='gdf')  # or 'gdf'
output

,id,name,population,level,geometry,settlement_name,district_name
0,0,Болото,111,Малое сельское поселение,"POLYGON ((3758260.573 8259102.313, 3758259.443...",Ленинградская область,Бокситогорский муниципальный район
1,1,Большой Остров,116,Малое сельское поселение,"POLYGON ((3761279.773 8283807.226, 3761278.618...",Ленинградская область,Бокситогорский муниципальный район
2,2,Бор,116,Малое сельское поселение,"POLYGON ((3761987.765 8283453.712, 3761986.610...",Ленинградская область,Бокситогорский муниципальный район
3,3,Бороватое,114,Малое сельское поселение,"POLYGON ((3760133.666 8276647.742, 3760132.521...",Ленинградская область,Бокситогорский муниципальный район
4,4,Бочево,108,Малое сельское поселение,"POLYGON ((3749169.369 8251663.313, 3749168.255...",Ленинградская область,Бокситогорский муниципальный район
...,...,...,...,...,...,...,...
2926,2926,Апраксин Бор,96,Малое сельское поселение,"POLYGON ((3477193.972 8217267.914, 3477192.922...",Ленинградская область,Тоснненский муниципальный район
2927,2927,Александровка,98,Малое сельское поселение,"POLYGON ((3486662.280 8220919.584, 3486661.219...",Ленинградская область,Тоснненский муниципальный район
2928,2928,Большая Горка,100,Малое сельское поселение,"POLYGON ((3503961.305 8244222.283, 3503960.233...",Ленинградская область,Тоснненский муниципальный район
2929,2929,Дроздово,101,Малое сельское поселение,"POLYGON ((3502629.676 8246185.212, 3502628.599...",Ленинградская область,Тоснненский муниципальный район


In [9]:
import matplotlib.colors as mcolors

def get_color_map(levels, base_colors):
    assert len(levels) == len(base_colors), "Количество уровней и базовых цветов должно совпадать"
    cmap = mcolors.LinearSegmentedColormap.from_list("heatmap", base_colors, N=len(levels))
    norm = mcolors.Normalize(vmin=0, vmax=len(levels) - 1)
    colors = [mcolors.to_hex(cmap(norm(i))) for i in range(len(levels))]
    return colors

def generate_sld():
    levels = ["Малое сельское поселение", "Среднее сельское поселение", "Большое сельское поселение", "Крупное сельское поселение", 
              "Малый город", "Средний город", "Большой город", "Крупный город", "Крупнейший город", "Сверхкрупный город"]
    
    # Используем указанные базовые цвета
    base_colors = ['#ffc100', '#ffa200', '#ff8400', '#ff6500', '#eb4704', '#d82a08', '#c40c0c', '#a7091f', '#890632', '#6c0345']
    level_colors = dict(zip(levels, get_color_map(levels, base_colors)))

    sld_template = '''
    <StyledLayerDescriptor version="1.0.0" xmlns="http://www.opengis.net/sld" xmlns:ogc="http://www.opengis.net/ogc" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd">
      <NamedLayer>
        <Name>towns</Name>
        <UserStyle>
          <Title>Town Levels</Title>
          <FeatureTypeStyle>
            {rules}
          </FeatureTypeStyle>
        </UserStyle>
      </NamedLayer>
    </StyledLayerDescriptor>
    '''
    
    rule_template = '''
    <Rule>
      <Name>{level}</Name>
      <Title>{level}</Title>
      <ogc:Filter>
        <ogc:PropertyIsEqualTo>
          <ogc:PropertyName>level</ogc:PropertyName>
          <ogc:Literal>{level}</ogc:Literal>
        </ogc:PropertyIsEqualTo>
      </ogc:Filter>
      <PolygonSymbolizer>
        <Fill>
          <CssParameter name="fill">{color}</CssParameter>
        </Fill>
        <Stroke>
          <CssParameter name="stroke">#000000</CssParameter>
          <CssParameter name="stroke-width">1</CssParameter>
        </Stroke>
      </PolygonSymbolizer>
    </Rule>
    '''

    rules = ''.join([rule_template.format(level=level, color=color) for level, color in level_colors.items()])
    sld = sld_template.format(rules=rules)
    
    return sld

def save_sld_to_file(sld, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(sld)

# Пример использования:
sld = generate_sld()
save_sld_to_file(sld, 'towns_style.sld')


In [31]:
import geopandas as gpd
import os

# Путь к директории с GeoJSON файлами
input_directory = '/Users/mvin/Desktop/Каркасы геосервер/Версия 2'
output_directory = '/Users/mvin/Desktop/Каркасы геосервер/Версия 2'

# Создаем выходную директорию, если ее нет
os.makedirs(output_directory, exist_ok=True)

# Получаем список всех файлов в директории
files = os.listdir(input_directory)

# Фильтруем только файлы с расширением .geojson
geojson_files = [file for file in files if file.endswith('.geojson')]

# Проходим по каждому файлу и конвертируем его в GPKG
for geojson_file in geojson_files:
    input_path = os.path.join(input_directory, geojson_file)
    output_path = os.path.join(output_directory, geojson_file.replace('.geojson', '.gpkg'))
    
    # Читаем GeoJSON файл
    gdf = gpd.read_file(input_path)
    
    # Сохраняем его в формате GPKG
    gdf.to_file(output_path, driver='GPKG')

print("Конвертация завершена!")

Конвертация завершена!


In [27]:
territories = gpd.read_file('/Users/mvin/Code/PopFrame/examples/karkas_graph_Moscow_obl.geojson')
territories

,name,level,population,geometry
0,Серебряные Пруды,Малый город,9358.0,POINT (38.72237 54.46798)
1,Узуново,Крупное сельское поселение,3417.0,POINT (38.61833 54.54061)
2,Благодать,Малое сельское поселение,52.0,POINT (38.62932 54.47175)
3,Ступино,Средний город,66463.0,POINT (38.07726 54.88660)
4,Кашира,Малый город,39929.0,POINT (38.15051 54.83745)
...,...,...,...,...
5724,None,Малое сельское поселение,NaN,"LINESTRING (35.50998 56.36097, 35.63585 56.37377)"
5725,None,Малое сельское поселение,NaN,"LINESTRING (35.62201 56.37238, 35.63585 56.37377)"
5726,None,Малое сельское поселение,NaN,"LINESTRING (35.60816 56.37178, 35.63585 56.37377)"
5727,None,Малое сельское поселение,NaN,"LINESTRING (35.68766 56.36313, 35.63585 56.37377)"


In [30]:
import matplotlib.colors as mcolors

def get_color_map(levels, base_colors):
    assert len(levels) == len(base_colors), "Количество уровней и базовых цветов должно совпадать"
    cmap = mcolors.LinearSegmentedColormap.from_list("heatmap", base_colors, N=len(levels))
    norm = mcolors.Normalize(vmin=0, vmax=len(levels) - 1)
    colors = [mcolors.to_hex(cmap(norm(i))) for i in range(len(levels))]
    return colors

def generate_sld_points():
    levels = ["Малое сельское поселение", "Среднее сельское поселение", "Большое сельское поселение", "Крупное сельское поселение", 
              "Малый город", "Средний город", "Большой город", "Крупный город", "Крупнейший город", "Сверхкрупный город"]
    
    # Используем указанные базовые цвета
    base_colors = ['#ffc100', '#ffa200', '#ff8400', '#ff6500', '#eb4704', '#d82a08', '#c40c0c', '#a7091f', '#890632', '#6c0345']
    level_colors = dict(zip(levels, get_color_map(levels, base_colors)))

    sld_template = '''
    <StyledLayerDescriptor version="1.0.0" xmlns="http://www.opengis.net/sld" xmlns:ogc="http://www.opengis.net/ogc" xmlns:xlink="http://www.opengis.net/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd">
      <NamedLayer>
        <Name>towns_points</Name>
        <UserStyle>
          <Title>Town Levels Points</Title>
          <FeatureTypeStyle>
            {rules}
          </FeatureTypeStyle>
        </UserStyle>
      </NamedLayer>
    </StyledLayerDescriptor>
    '''
    
    rule_template = '''
    <Rule>
      <Name>{level}</Name>
      <Title>{level}</Title>
      <ogc:Filter>
        <ogc:PropertyIsEqualTo>
          <ogc:PropertyName>level</ogc:PropertyName>
          <ogc:Literal>{level}</ogc:Literal>
        </ogc:PropertyIsEqualTo>
      </ogc:Filter>
      <PointSymbolizer>
        <Graphic>
          <Mark>
            <WellKnownName>circle</WellKnownName>
            <Fill>
              <CssParameter name="fill">{color}</CssParameter>
            </Fill>
            <Stroke>
              <CssParameter name="stroke">#000000</CssParameter>
              <CssParameter name="stroke-width">1</CssParameter>
            </Stroke>
          </Mark>
          <Size>6</Size>
        </Graphic>
      </PointSymbolizer>
    </Rule>
    '''

    rules = ''.join([rule_template.format(level=level, color=color) for level, color in level_colors.items()])
    sld = sld_template.format(rules=rules)
    
    return sld

def generate_sld_lines():
    levels = ["Малое сельское поселение", "Среднее сельское поселение", "Большое сельское поселение", "Крупное сельское поселение", 
              "Малый город", "Средний город", "Большой город", "Крупный город", "Крупнейший город", "Сверхкрупный город"]
    
    # Используем указанные базовые цвета
    base_colors = ['#ffc100', '#ffa200', '#ff8400', '#ff6500', '#eb4704', '#d82a08', '#c40c0c', '#a7091f', '#890632', '#6c0345']
    level_colors = dict(zip(levels, get_color_map(levels, base_colors)))

    sld_template = '''
    <StyledLayerDescriptor version="1.0.0" xmlns="http://www.opengis.net/sld" xmlns:ogc="http://www.opengis.net/ogc" xmlns:xlink="http://www.opengis.net/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd">
      <NamedLayer>
        <Name>towns_lines</Name>
        <UserStyle>
          <Title>Town Levels Lines</Title>
          <FeatureTypeStyle>
            {rules}
          </FeatureTypeStyle>
        </UserStyle>
      </NamedLayer>
    </StyledLayerDescriptor>
    '''
    
    rule_template = '''
    <Rule>
      <Name>{level}</Name>
      <Title>{level}</Title>
      <ogc:Filter>
        <ogc:PropertyIsEqualTo>
          <ogc:PropertyName>level</ogc:PropertyName>
          <ogc:Literal>{level}</ogc:Literal>
        </ogc:PropertyIsEqualTo>
      </ogc:Filter>
      <LineSymbolizer>
        <Stroke>
          <CssParameter name="stroke">{color}</CssParameter>
          <CssParameter name="stroke-width">2</CssParameter>
        </Stroke>
      </LineSymbolizer>
    </Rule>
    '''

    rules = ''.join([rule_template.format(level=level, color=color) for level, color in level_colors.items()])
    sld = sld_template.format(rules=rules)
    
    return sld

def save_sld_to_file(sld, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(sld)

# Пример использования:
sld_points = generate_sld_points()
save_sld_to_file(sld_points, 'towns_points_style.sld')

sld_lines = generate_sld_lines()
save_sld_to_file(sld_lines, 'towns_lines_style.sld')
